# Invasive Devices
Feature Engineering: Invasive Devices



Load libraries and setup environment

In [3]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

MIMIC BigQuery access

In [4]:
# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# authenticate
auth.authenticate_user()

# MIMIC project ID
project_id = "ml-health-395010"

MIMIC IV setup (MIMIC IV version 2.2)

In [5]:
# CODE TAKEN FROM TUTORIAL 3
# accessed at: 14-08-2023

# Set up environment variables
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'


## Central Line-Associated Bloodstream Infection (CLABSI)

In [13]:
clabsi = """
SELECT
    mv.stay_id,
    mv.itemid,
    CASE
        WHEN di.label IN ('Arterial Line', 'A-Line') THEN 'Arterial'
        WHEN di.label IN ('CCO PA Line', 'CCO PAC') THEN 'Continuous Cardiac Output PA'
        WHEN di.label IN ('Dialysis Catheter', 'Dialysis Line') THEN 'Dialysis'
        WHEN di.label IN ('Hickman', 'Tunneled (Hickman) Line') THEN 'Hickman'
        WHEN di.label IN ('IABP', 'IABP line') THEN 'IABP'
        WHEN di.label IN ('Multi Lumen', 'Multi-lumen') THEN 'Multi Lumen'
        WHEN di.label IN ('PA Catheter', 'PA line') THEN 'PA'
        WHEN di.label IN ('PICC Line', 'PICC line') THEN 'PICC'
        WHEN di.label IN ('Pre-Sep Catheter', 'Presep Catheter') THEN 'Pre-Sep'
        WHEN di.label IN ('Trauma Line', 'Trauma line') THEN 'Trauma'
        WHEN di.label IN ('Triple Introducer', 'TripleIntroducer') THEN 'Triple Introducer'
        WHEN di.label IN ('Portacath', 'Indwelling Port (PortaCath)') THEN 'Portacath'
        ELSE di.label
    END AS type,
    CASE
        WHEN mv.location IN ('Left Antecub', 'Left Antecube') THEN 'Left Antecube'
        WHEN mv.location IN ('Left Axilla', 'Left Axilla.') THEN 'Left Axilla'
        WHEN mv.location IN ('Left Brachial', 'Left Brachial.') THEN 'Left Brachial'
        WHEN mv.location IN ('Left Femoral', 'Left Femoral.') THEN 'Left Femoral'
        WHEN mv.location IN ('Right Antecub', 'Right Antecube') THEN 'Right Antecube'
        WHEN mv.location IN ('Right Axilla', 'Right Axilla.') THEN 'Right Axilla'
        WHEN mv.location IN ('Right Brachial', 'Right Brachial.') THEN 'Right Brachial'
        WHEN mv.location IN ('Right Femoral', 'Right Femoral.') THEN 'Right Femoral'
        ELSE mv.location
    END AS location,
    mv.locationcategory,
    mv.starttime,
    mv.endtime,
    mv.value,
    mv.valueuom
FROM `physionet-data.mimiciv_icu.procedureevents` mv
INNER JOIN `physionet-data.mimiciv_icu.d_items` di
ON mv.itemid = di.itemid
WHERE mv.itemid IN
(
    227719 -- AVA Line
    , 225752 -- Arterial Line
    , 224269 -- CCO PAC
    , 224267 -- Cordis/Introducer
    , 224270 -- Dialysis Catheter
    , 224272 -- IABP line
    , 226124 -- ICP Catheter
    , 228169 -- Impella Line
    , 225202 -- Indwelling Port (PortaCath)
    , 228286 -- Intraosseous Device
    , 225204 -- Midline
    , 224263 -- Multi Lumen
    , 224560 -- PA Catheter
    , 224264 -- PICC Line
    , 225203 -- Pheresis Catheter
    , 224273 -- Presep Catheter
    , 225789 -- Sheath
    , 225761 -- Sheath Insertion
    , 228201 -- Tandem Heart Access Line
    , 228202 -- Tandem Heart Return Line
    , 224268 -- Trauma line
    , 225199 -- Triple Introducer
    , 225315 -- Tunneled (Hickman) Line
    , 225205 -- RIC
)
AND mv.location is not null   --- !!!!! COMMENT THIS LINE WHEN EXTRATING FEATURE
LIMIT 1000
;
"""

In [14]:
run_query(clabsi)

,stay_id,itemid,type,location,locationcategory,starttime,endtime,value,valueuom
0,38934988,226124,ICP Catheter,Lumbar,ICP Line,2179-07-14 15:33:00,2179-07-15 11:30:00,1197.0,min
1,38427719,226124,ICP Catheter,Lumbar,ICP Line,2156-04-13 23:19:00,2156-04-18 17:00:00,6821.0,min
2,37277638,226124,ICP Catheter,Lumbar,ICP Line,2189-10-29 16:16:00,2189-10-31 14:00:00,2744.0,min
3,33018681,226124,ICP Catheter,Lumbar,ICP Line,2173-10-05 19:30:00,2173-10-07 08:05:00,2195.0,min
4,32944560,226124,ICP Catheter,Lumbar,ICP Line,2171-04-04 18:13:00,2171-04-09 17:15:00,7142.0,min
...,...,...,...,...,...,...,...,...,...
995,33589102,224263,Multi Lumen,Left IJ,Invasive Venous,2128-08-24 02:11:00,2128-08-27 15:01:00,5090.0,min
996,38377313,224263,Multi Lumen,Left IJ,Invasive Venous,2179-10-29 21:30:00,2179-11-02 12:00:00,5190.0,min
997,38080928,224263,Multi Lumen,Left IJ,Invasive Venous,2168-08-05 04:45:00,2168-08-07 00:32:00,2627.0,min
998,39367199,224263,Multi Lumen,Left IJ,Invasive Venous,2173-05-25 16:01:00,2173-05-26 15:01:00,1380.0,min


Note:
- The value is redundat as it calculates the duration of the procedure

## Ventilator-Associated Pneumonia (VAP)

In [27]:
vap = """
SELECT DISTINCT
  pe.stay_id,
  pe.itemid,
  ve.ventilation_status as type,
  pe.location,
  pe.locationcategory,
  pe.starttime,
  pe.endtime,
  pe.value,
  pe.valueuom
FROM
  `physionet-data.mimiciv_derived.ventilation` ve
  INNER JOIN `physionet-data.mimiciv_icu.procedureevents` pe
    ON pe.stay_id = ve.stay_id
  INNER JOIN `physionet-data.mimiciv_icu.d_items` di
    ON pe.itemid = di.itemid
WHERE
  di.label = 'Invasive Ventilation'
  AND ve.ventilation_status <> 'NonInvasiveVent'
LIMIT
  1000
;
"""

In [28]:
run_query(vap)

,stay_id,itemid,type,location,locationcategory,starttime,endtime,value,valueuom
0,37510196,225792,InvasiveVent,None,None,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,min
1,37510196,225792,SupplementalOxygen,None,None,2131-01-11 04:40:00,2131-01-12 17:40:00,2220.0,min
2,37510196,225792,InvasiveVent,None,None,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,min
3,37510196,225792,SupplementalOxygen,None,None,2131-01-13 04:00:00,2131-01-19 17:45:00,9465.0,min
4,37510196,225792,InvasiveVent,None,None,2131-01-15 04:07:00,2131-01-19 17:43:00,6576.0,min
...,...,...,...,...,...,...,...,...,...
995,33514285,225792,SupplementalOxygen,None,None,2112-06-26 13:23:00,2112-06-26 15:58:00,155.0,min
996,32536920,225792,InvasiveVent,None,None,2170-06-12 12:36:00,2170-06-13 11:00:00,1344.0,min
997,32536920,225792,SupplementalOxygen,None,None,2170-06-12 12:36:00,2170-06-13 11:00:00,1344.0,min
998,32893819,225792,InvasiveVent,None,None,2174-09-30 18:33:00,2174-10-02 09:55:00,2362.0,min


Note:
- There is duplicate ventilation for the same icu stay
- The value is redundat as it calculates the duration of the procedure
- If needed can add information from ventilator setting (chartevent)
- The location and location category is all null

## Catheter-Associated Urinary Tract Infection (CAUTI)

In [29]:
cauti = """
SELECT
  pe.stay_id,
  pe.itemid,
  di.label as type,
  pe.location,
  pe.locationcategory,
  pe.starttime,
  pe.endtime,
  pe.value,
  pe.valueuom
FROM
  `physionet-data.mimiciv_icu.procedureevents` pe
  INNER JOIN `physionet-data.mimiciv_icu.d_items` di
    ON pe.itemid = di.itemid
WHERE
pe.ordercategoryname = 'Tubes'
LIMIT
  1000
;
"""

In [30]:
run_query(cauti)

,stay_id,itemid,type,location,locationcategory,starttime,endtime,value,valueuom
0,36753294,229351,Foley Catheter,None,None,2141-05-22 20:18:00,2141-05-27 22:24:00,7326.0,min
1,34577403,229351,Foley Catheter,None,None,2137-08-12 21:11:00,2137-08-13 17:55:00,1244.0,min
2,30213599,229351,Foley Catheter,None,None,2159-02-20 18:30:00,2159-02-24 12:10:00,5380.0,min
3,35009126,229351,Foley Catheter,None,None,2186-11-12 18:02:00,2186-11-17 21:16:00,7394.0,min
4,32104791,229351,Foley Catheter,None,None,2168-04-12 12:34:00,2168-04-14 21:19:00,3405.0,min
...,...,...,...,...,...,...,...,...,...
995,31759478,229351,Foley Catheter,None,None,2176-07-16 18:15:00,2176-07-16 18:29:00,14.0,min
996,32822335,229351,Foley Catheter,None,None,2176-07-16 19:00:00,2176-07-20 15:01:00,5521.0,min
997,32008270,229351,Foley Catheter,None,None,2147-09-16 10:16:00,2147-09-20 13:09:00,5933.0,min
998,39420552,229351,Foley Catheter,None,None,2147-09-14 18:59:00,2147-09-16 05:07:00,2048.0,min


Note:
- In procedureevent only foley cathether is recorded
- The value is redundat as it calculates the duration of the procedure
- If needed can add information from urine output (outputevent)
- The location and location category is all null